In [1]:
import pandas_ta as ta
import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import config_api
import matplotlib.pyplot as plt

In [2]:
user = config_api.tradingview_user
password = config_api.tradingview_password

In [3]:
tv = TvDatafeed(user, password, chromedriver_path= None)

Message: element not interactable
  (Session info: headless chrome=98.0.4758.82)
, ('element not interactable\n  (Session info: headless chrome=98.0.4758.82)', None, None)
automatic login failed
 Reinitialize tvdatafeed with auto_login=False 


######### (symbol: str, exchange: str = 'NSE', interval: Interval = Interval.in_daily, n_bars: int = 10, fut_contract: int | None = None, extended_session: bool = False) -> DataFrame)


In [4]:
bnb_1d = tv.get_hist(symbol= 'BNBUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)
btc_1d = tv.get_hist(symbol= 'BTCUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)
eth_1d = tv.get_hist(symbol= 'ETHUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)
xrp_1d = tv.get_hist(symbol= 'XRPUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)
ada_1d = tv.get_hist(symbol= 'ADAUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)

In [5]:
axs_1d = tv.get_hist(symbol= 'AXSUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars= 5000)

In [6]:
# set var of data here

In [7]:
df = eth_1d.copy()

In [8]:
# config strategy

In [9]:
rsi_bm = 50
obv_fast = 5
obv_slow = 35
ma_type = 'ema'
ma_fast = 5
ma_slow = 15

In [10]:
df['rsi'] = df.ta.rsi(length=14)
ma_obv = df.ta.ma_obv(fast= obv_fast, slow= obv_slow, ma_type= 'ema', append= True)
ma_cross = df.ta.ma_cross(fast= ma_fast, slow= ma_slow, ma_type= 'ema', append= True)

In [11]:
### strategy name will be in strat variable

In [12]:
symbol_str = f'{df.symbol}'.split()[3].split(':')[1] # เรียก col symbol มา และสั่งแยกเอา เอาตัวที่4 แยกออก เอาตัว หลัง :
strat = symbol_str + '_' + ma_obv.name + '_rsi50' + '_' + ma_cross.name  # รวมให้เป็นชื่อกลยุทธ์รวม

In [13]:
df.loc[df['rsi'] >= rsi_bm, 'strength'] = 1
df.loc[df['rsi'] < rsi_bm, 'strength'] = -1
df.loc[df[f'{ma_obv.name}'] > 0, 'vol_in'] = 1
df.loc[df[f'{ma_obv.name}'] < 0, 'vol_in'] = -1
df.loc[df[f'{ma_cross.name}'] > 0, 'ma_cross'] = 1
df.loc[df[f'{ma_cross.name}'] < 0, 'ma_cross'] = -1


In [14]:
# สร้าง signal ที่จะ feed ให้กับ vector_bt ทำเป็นตัว entry/exit
# df['signal'] = (df['vol_in'] == 1) & (df['strength'] == 1) & (df['SUPERTd_7_3.0'] == 1)
# df['signal'] = (df['strength'] == 1) & (df['SUPERTd_7_5.0'] == 1)

#### volin and strength
#df['signal'] = (df['vol_in'] == 1) & (df['strength'] == 1)

#### vol_in, strength , ma_cross
df['signal'] = (df['vol_in'] == 1) & (df['strength'] == 1) & (df['ma_cross'] == 1)

## อนาคตหน้าพวก secondary signal (ตัวindy ต่างๆที่รวมเป็น signal ใหญ่) มีหลาย label ----> อาจจะใช้เป็นวิธี vol_in > 2 ไรงี้

In [27]:
df_signal = df[['open', 'close', 'rsi', f'{ma_obv.name}', 'vol_in', 'strength', 'ma_cross', 'signal']] # 'vol_in'

In [29]:
df_signal['next_open'] = df_signal['open'].shift(-1)
df_signal = df_signal.dropna()
signal_vectorbt = df.ta.tsignals(df_signal['signal'], asbool= True, append= True)

<ipython-input-29-88eb26ab93f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_signal['next_open'] = df_signal['open'].shift(-1)


In [30]:
df_signal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1606 entries, 2017-09-20 07:00:00 to 2022-02-11 07:00:00
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   open               1606 non-null   float64
 1   close              1606 non-null   float64
 2   rsi                1606 non-null   float64
 3   MA_OBVe_5_35_diff  1606 non-null   float64
 4   vol_in             1606 non-null   float64
 5   strength           1606 non-null   float64
 6   ma_cross           1606 non-null   float64
 7   signal             1606 non-null   bool   
 8   next_open          1606 non-null   float64
dtypes: bool(1), float64(8)
memory usage: 179.0 KB


In [31]:
signal_vectorbt.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1606 entries, 2017-09-20 07:00:00 to 2022-02-11 07:00:00
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   TS_Trends   1606 non-null   bool 
 1   TS_Trades   1606 non-null   int32
 2   TS_Entries  1606 non-null   bool 
 3   TS_Exits    1606 non-null   bool 
dtypes: bool(3), int32(1)
memory usage: 88.1 KB


In [32]:
port = vbt.Portfolio.from_signals(
    df_signal['next_open'],
    entries= signal_vectorbt['TS_Entries'],
    exits= signal_vectorbt['TS_Exits'],
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)

In [33]:
port.plot().show()

In [34]:
port.stats()

Start                                2017-09-20 07:00:00
End                                  2022-02-11 07:00:00
Period                                1606 days 00:00:00
Start Value                                        100.0
End Value                                    1524.718644
Total Return [%]                             1424.718644
Benchmark Return [%]                          938.134752
Max Gross Exposure [%]                             100.0
Total Fees Paid                                82.601275
Max Drawdown [%]                               61.224023
Max Drawdown Duration                  975 days 00:00:00
Total Trades                                          41
Total Closed Trades                                   41
Total Open Trades                                      0
Open Trade P&L                                       0.0
Win Rate [%]                                   34.146341
Best Trade [%]                                175.959454
Worst Trade [%]                

In [40]:
port.drawdowns.plot()

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'showlegend': False,
              't…

In [58]:
port.plot_underwater()


FigureWidget({
    'data': [{'fill': 'tozeroy',
              'fillcolor': 'rgba(220,57,18,0.3000)',
         …

In [36]:
port.annual_returns()

datetime
2017-09-20    2.054811
2018-09-20   -0.158090
2019-09-20   -0.187552
2020-09-19    4.794703
2021-09-19    0.259252
Freq: 365D, dtype: float64

In [37]:
return_adjust_vol = port.annualized_return() / port.annualized_volatility()
ret_adj_risk = ['Annualized Return Adjust Vol', return_adjust_vol]
ret_adj_risk


['Annualized Return Adjust Vol', 1.317779297122562]

## Save and write to file zone

In [93]:
port.plot().write_image(file= f'../trading_signal_backtest/port_plot/{strat}.png', format= 'png', scale= 2.0)

In [98]:
# strat is strategy string variable
port_df = pd.DataFrame(port.stats()).rename(columns={0: strat})
port_df = port_df.reset_index()
series = pd.Series(ret_adj_risk, index= port_df.columns)
port_df = port_df.append(series, ignore_index= True)
port_df = port_df.set_index('index')


In [99]:
### only first time ( creating csv )
#port_df.to_csv('crypto_tf1d_backtest.csv', index= True)

In [100]:
#### if csv file is exist use this script
port_csv = pd.read_csv('./crypto_tf1d_backtest.csv', index_col=0)
port_csv[f'{strat}'] = port_df
port_csv.to_csv('crypto_tf1d_backtest.csv', index= True)
#### save portfolio records to cache in the same name as column in csv
port.save(f'{strat}')

In [776]:
### for load backtest save from file
#pf = vbt.Portfolio.load(f'{strat}')
pf = vbt.Portfolio.load('XRPUSDT_IBVe_5_35_diff_rsi50')
pf.stats()

Start                                2018-05-04 07:00:00
End                                  2022-01-30 07:00:00
Period                                1368 days 00:00:00
Start Value                                        100.0
End Value                                     101.977829
Total Return [%]                                1.977829
Benchmark Return [%]                          -30.602383
Max Gross Exposure [%]                             100.0
Total Fees Paid                                 18.22642
Max Drawdown [%]                               85.437046
Max Drawdown Duration                  926 days 00:00:00
Total Trades                                          61
Total Closed Trades                                   61
Total Open Trades                                      0
Open Trade P&L                                       0.0
Win Rate [%]                                    22.95082
Best Trade [%]                                124.931259
Worst Trade [%]                

In [415]:
port.stats(tags= ['trades'])

Total Trades                                          61
Total Closed Trades                                   61
Total Open Trades                                      0
Open Trade P&L                                       0.0
Win Rate [%]                                   27.868852
Best Trade [%]                                464.859706
Worst Trade [%]                               -18.436788
Avg Winning Trade [%]                          70.078664
Avg Losing Trade [%]                           -5.959433
Avg Winning Trade Duration    36 days 02:49:24.705882353
Avg Losing Trade Duration      4 days 22:21:49.090909090
Profit Factor                                   3.595367
Expectancy                                     64.608748
dtype: object

In [422]:
port_csv

,BINANCE:BNBUSDT_obv20f40s_rsi>50,BINANCE:BTCUSDT_obv20f40s_rsi>50,BINANCE:ETHUSDT_obv20f40s_rsi>50,BINANCE:XRPUSDT_obv20f40s_rsi>50,BINANCE:ADAUSDT_obv20f40s_rsi>50,BINANCE:BNBUSDT_obv10f40s_rsi>50
index,,,,,,
Start,2017-11-06 07:00:00,2017-08-17 07:00:00,2017-08-17 07:00:00,2018-05-04 07:00:00,2018-04-17 07:00:00,2017-11-06 07:00:00
End,2022-01-30 07:00:00,2022-01-30 07:00:00,2022-01-30 07:00:00,2022-01-30 07:00:00,2022-01-30 07:00:00,2022-01-30 07:00:00
Period,1547 days 00:00:00,1628 days 00:00:00,1628 days 00:00:00,1368 days 00:00:00,1385 days 00:00:00,1547 days 00:00:00
Start Value,100.0,100.0,100.0,100.0,100.0,100.0
End Value,3990.4013272781935,763.0893333086645,767.7514002627123,88.58654409933686,925.5403969754994,4041.133647
Total Return [%],3890.401327278194,663.0893333086645,667.7514002627123,-11.413455900663138,825.5403969754993,3941.133647
Benchmark Return [%],24132.97262889879,785.2875558916052,766.2649006622516,-31.374311720418017,333.2234130255564,24686.760025
Max Gross Exposure [%],100.0,100.0,100.0,100.0,100.0,100.0
Total Fees Paid,231.81822229254496,61.61804132938498,46.25945348095007,11.76837639292209,64.8453051604172,198.381372


In [367]:
port.get_drawdowns().plot().show()

In [366]:
port.plot_underwater()

FigureWidget({
    'data': [{'fill': 'tozeroy',
              'fillcolor': 'rgba(220,57,18,0.3000)',
         …